In [1]:
!pip install pandas
!pip install folium

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd
import folium

In [64]:
events = pd.read_csv("data/events.csv")
people = pd.read_csv("data/people.csv")
locations = pd.read_csv("data/locations.csv")
occasions = pd.read_csv("data/occasions.csv")
event_participants = pd.read_csv("data/event_participants.csv")
event_locations = pd.read_csv("data/event_locations.csv")
associations = pd.read_csv("data/associations.csv")

In [65]:
assocs = associations.set_index("Association_ID")["Name"].to_dict()
assocs

{'ASSOC:00001': 'Catholic',
 'ASSOC:00002': 'Protestant',
 'ASSOC:00003': 'Imperial',
 'ASSOC:00004': 'Regensburg'}

In [66]:
assoc_colours = {"Catholic": "orange", "Protestant": "blue", "Imperial": "black", "Regensburg": "red", "undefined": "lightgray"}

In [67]:
def get_occasion(event_key, event_df, occ_df):
    occasion_key = event_df.loc[event_df["Event_ID"] == event_key, "Occasion"].iloc[0]
    if isinstance(occasion_key, str):
        occasion = occ_df.loc[occ_df["Occasion_ID"] == occasion_key].to_dict("records")[0]
        if isinstance(occasion["Association"], str):
            occasion["Association"] = assocs[occasion["Association"]]
        else:
            occasion["Association"] = "undefined"
    else:
        occasion = {'Occasion_ID': 'OCC:00000', 'Names': 'undefined',
                    'Association': 'undefined', 'Description': ''}
    return occasion

In [68]:
def get_participants(event_key, part_df, people_df):
    selection = part_df.loc[part_df['Event_key'] == event_key]
    selection_tuple = list(zip(selection.Person_key, selection.Certainty))
    part_proper = []
    for i in selection_tuple:
        person = people_df.loc[people_df["Person_ID"] == i[0]].to_dict('records')[0]
        if isinstance(person["Association"], str):
            person["Association"] = assocs[person["Association"]]
        else:
            person["Association"] = "undefined"
        part_proper.append((person, i[1]))
    
    return part_proper


In [69]:
def get_event_locations(event_key, ev_loc_df, loc_df):
    selection = ev_loc_df.loc[ev_loc_df["Event_key"] == event_key]
    selection_tuple = list(zip(selection.Location_key, 
                            selection.Number, 
                            selection.Certainty))
    locs_proper = []
    for i in selection_tuple:
        loc = loc_df.loc[loc_df["Location_ID"] == i[0]].to_dict('records')[0]
        coordinates = tuple([float(num) for num in loc["Coordinates"].split(", ")])
        loc["Coordinates"] = coordinates
        if isinstance(loc["Association"], str):
            loc["Association"] = assocs[loc["Association"]]
        else:
            loc["Association"] = "undefined"
        locs_proper.append((loc, i[1], i[2]))
    return locs_proper

In [70]:
class Event:
    def __init__(self, id, name, 
                 occasion, date_start, date_end, 
                 type, participants, locations, certainty):
        self.id = id
        self.name = name
        self.occasion = occasion
        self.start = date_start
        self.end = date_end
        self.type = type
        self.participants = participants
        self.locations = locations
        self.certainty = certainty

    def list_participants(self):
        return [participant[0] for participant in self.participants]
    
    def min_number_of_people(self):
        return sum([int(participant[0]["Amount"].replace(">", "")) for participant in self.participants])

    def occasion_assoc(self):
        if self.occasion == "undefined":
            return "undefined"
        else:
            return self.occasion["Association"]
    
    def min_part_assocs(self):
        participant_assocs = []
        for i in self.participants:
            participant_assocs.append({"number": int(i[0]["Amount"].replace(">", "")), "assoc": i[0]["Association"]})
        
        data = pd.DataFrame.from_dict(participant_assocs)

        return data.groupby('assoc')["number"].sum().to_dict()
    
    def get_waypoints(self):
        waypoints = []
        for i in self.locations:
            waypoints.append({"name": i[0]["Names"].split(";")[0], 
                              "coordinates": i[0]["Coordinates"],
                              "association": i[0]["Association"],
                              "gnd": i[0]["GND"],
                              "type": i[0]["Type"],
                              "number": i[1],
                              "certainty": i[2]})
        return waypoints

In [71]:
all_events = []

for index, row in events.iterrows():
    id = row["Event_ID"]
    name = row["Name"]
    occasion = get_occasion(id, events, occasions)
    start = row["Date_start"]
    end = row["Date_end"]
    event_type = row["Type"]
    participants_list = get_participants(id, event_participants, people)
    event_locations_list = get_event_locations(id, event_locations, locations)
    certainty = row["Certainty"]

    event = Event(id, name, occasion, start, 
                  end, event_type, participants_list, 
                  event_locations_list, certainty)

    all_events.append(event)

In [72]:
event_names = [x.name for x in all_events]

In [73]:
events_named = {}
for name, event_object in zip(event_names, all_events):
    events_named[name] = event_object

In [74]:
def get_map_bounds(waypoints: list) -> tuple:
    all_coordinates = [x["coordinates"] for x in waypoints]
    df = pd.DataFrame(all_coordinates)
    sw = df.min().values.tolist()
    ne = df.max().values.tolist()
    return (sw, ne)

In [75]:
# get maximum coordinates of map
all_coord = locations["Coordinates"].to_list()
coords = []
for i in all_coord:
    coords.append(tuple([float(num) for num in i.split(", ")]))

df = pd.DataFrame(coords)
sw = df.min().values.tolist()
ne = df.max().values.tolist()
max_coords = (sw, ne)

In [76]:
# create a map for all events
regensburg_layered = folium.Map(location=[49.018,12.09], zoom_start=14)

for event in all_events:
    #print(event.name)
    
    feature_group = folium.FeatureGroup(f"{event.name}")
    
    waypoints = event.get_waypoints()
    
    for point in waypoints:

        if isinstance(point["gnd"], str):
            marker_tooltip = f'<a href="https://explore.gnd.network/gnd/{point["gnd"]}" target="_blank">{point["name"]}</a>'
        else:
            marker_tooltip = point["name"]

        # set icon customisation depending on waypoint

        participants = event.min_number_of_people()
        occasion = event.occasion["Names"]
        occ_assoc = event.occasion_assoc()

        sign = "home"
        color = assoc_colours[point["association"]]
        opacity = round((1 / int(point["certainty"])) + 0.1, 1)
        
        arrow_size = 1 + (participants - 1) * (10 - 1) / (5000 - 1)

        custom_icon = folium.Icon(color=color, 
                                  icon=sign, 
                                  prefix='glyphicon', 
                                  icon_color='white')

        folium.Marker(location = point['coordinates'], 
                      popup=marker_tooltip, 
                      icon=custom_icon, opacity=opacity
                     ).add_to(feature_group)

    line_tooltip = f'<b>Name:</b> {event.name}<br><b>Anlass:</b> {occasion}</b><br><b>Min. Anzahl an Teilnehmern:</b> {event.min_number_of_people()}.'
    for start, end in zip(waypoints, waypoints[1:]):
        folium.PolyLine([start["coordinates"], 
                         end["coordinates"]],
                        color=assoc_colours[occ_assoc],
                        weight=arrow_size,
                        opacity=1,
                        tooltip=line_tooltip
                       ).add_to(feature_group)
    
    feature_group.add_to(regensburg_layered)
    
folium.LayerControl().add_to(regensburg_layered)

regensburg_layered.fit_bounds(max_coords)
regensburg_layered